In [1]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 27.7 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 3.6 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.6-cp310-cp310-linux_x86_64.whl size=293632 sha256=0a73a078d3ee04c5893f7274c9b27b13e3c99c0eff72e6d63e3369f385fa7f6c
  Stored in directory: /root/.cache/pip/wheels/4b/b9/b0/83d6e258e8f963f5ff111a2cd8c483ca59372a86e6a2535212
Successfully built peewee


In [6]:
!pip install databento

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.5 MB/s eta 0:00:0000:01


In [4]:
import os
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from tqdm import tqdm
import datetime
from datetime import datetime as dt, timedelta as td

import sklearn
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import Model, Sequential, Input
from tensorflow.keras.layers import LSTM, GRU, Dense, Conv2D, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError

import warnings
warnings.filterwarnings("ignore")

In [7]:
import yfinance as yf
import databento as db

In [21]:
ibm =  yf.download("IBM", start="1962-07-01", end=dt.today())
ibm.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1962-07-02,4.324410,4.461440,4.314850,4.461440,0.926255,1459170
1962-07-03,4.461440,4.509242,4.435946,4.480561,0.930225,2706525
1962-07-05,4.480561,4.499681,4.445507,4.461440,0.926255,2110305
1962-07-06,4.410453,4.410453,4.359465,4.397706,0.913023,1702365
1962-07-09,4.397706,4.467814,4.346718,4.464627,0.926917,1976940


In [25]:
class DataLoader:
    """
    """
    def __init__(self, universe):
        self.universe = universe

        
class YahooDataLoader(DataLoader):
    """
    """
    def __init__(self, universe):
        super().__init__(universe)
        
    def load_data(self, ticker='IBM', start_date='2010-01-01', end_date=dt.today(), seq_length=100):
        """
        """
        # Fetch data from Yahoo Finance
        data = yf.download(ticker, start=start_date, end=end_date)
        self.raw_data = data

        # Assuming the stock returns are calculated from adjusted close prices
        returns = data['Adj Close'].pct_change().dropna().values
        self.raw_returns = returns

        # Normalize data
        min_val = np.min(returns)
        max_val = np.max(returns)
        returns = (returns - min_val) / (max_val - min_val)

        # Create sequences
        sequences = []
        for i in range(len(returns) - seq_length):
            sequences.append(returns[i:i + seq_length])

        sequences = np.array(sequences)
        self.sequences = sequences
        self.min, self.max = min_val, max_val
        return sequences, min_val, max_val
    
    def get_raw_data(self):
        return self.raw_data
    
    def get_raw_returns(self):
        return self.raw_returns
    
    def get_sequences(self):
        return self.sequences

In [26]:
# Yahoo Finnace Data
yfdataloader = YahooDataLoader(universe="")

sequences, min_val, max_val = yfdataloader.load_data(ticker='IBM', 
                                                     start_date='1962-07-01', 
                                                     end_date=dt.today())

[*********************100%***********************]  1 of 1 completed


In [28]:
yfdataloader.get_raw_returns()

array([ 0.00428675, -0.00426845, -0.01428528, ..., -0.00135857,
        0.00221701,  0.01623937])

In [29]:
sequences

array([[0.65282987, 0.62950732, 0.60220018, ..., 0.67320561, 0.64748173,
        0.71882467],
       [0.62950732, 0.60220018, 0.68262838, ..., 0.64748173, 0.71882467,
        0.64641393],
       [0.60220018, 0.68262838, 0.7072999 , ..., 0.71882467, 0.64641393,
        0.68321854],
       ...,
       [0.65855   , 0.65094471, 0.63381919, ..., 0.64309127, 0.66727872,
        0.65147093],
       [0.65094471, 0.63381919, 0.59405447, ..., 0.66727872, 0.65147093,
        0.63744001],
       [0.63381919, 0.59405447, 0.63909219, ..., 0.65147093, 0.63744001,
        0.64718749]])

In [30]:
class Generator(Model):
    """
    Vanilla Generator class.
    """
    def __init__(self, hidden_dim):
        super(Generator, self).__init__()
        self.hidden_dim = hidden_dim
        self.build()
    
    def build(self, input_shape):
        model = Sequential(name="Generator")
        model.add(
            LSTM(units=self.hidden_dim, return_sequences=True),
            LSTM(units=self.hidden_dim, return_sequences=True),
            LSTM(units=self.hidden_dim, return_sequences=True),
            Dense(units=self.hidden_dim, activation="sigmoid", name="Generator_Output")
        )
        self.model = model
        return model
    
    def call(self, inputs):
        return self.model(inputs)

    
class Discriminator(Model):
    """
    Vanilla Discriminator class.
    """
    def __init__(self, hidden_dim):
        super(Discriminator, self).__init__()
        self.hidden_dim = hidden_dim
        
    def build(self, input_shape):
        model = Sequential(name="Discriminator")
        model.add(
            LSTM(units=self.hidden_dim, return_sequences=True),
            LSTM(units=self.hidden_dim, return_sequences=True),
            LSTM(units=self.hidden_dim, return_sequences=True),
            Dense(units=1, activation="sigmoid", name="Discriminator_Output")
        )

In [ ]:
class GAN():
    """
    Vanilla Generative Adversarial Network class.
    """
    def __init__(self, params, hidden_dim, seq_len, n_seq, gamma):
        """
        """
        self.seq_len = seq_len
        self.n_seq = n_seq
        self.hidden_dim = hidden_dim
        self.gamma = gamma
        
        self.generator = Generator()
        self.discriminator = Discriminator()
        
    def train(self, epochs):
        ...